## Making YouTube Titles More Clickbait

### By: Jacob DeMuth

##### Goal: Make YouTube Video Titles "more appealing" to viewers
##### Subgoal: Make Calix's Video Titles Better

##### Note: I used GPT2-Large which is the 774M parameter of GPT 2

## Install Libraries

In [3]:
## !pip install transformers
## !pip install wandb
## !pip install trl
## !pip install pandas
## !pip install datasets
## !pip install accelerate
## !pip install tyro
## !pip install nltk -U

## Setup Stuff

In [4]:
import torch
from tqdm import tqdm
import pandas as pd
import wandb
import os

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler


In [5]:

config = PPOConfig(
    model_name    = "openai-community/gpt2-large",
    learning_rate = 1.41e-5,
    ## log_with      = "wandb",
)

sent_kwargs = {
         "return_all_scores": True, 
         "function_to_apply": "none", 
         "batch_size": 16
}


In [6]:
wandb.init(mode="disabled") 
os.environ['WANDB_DISABLED'] = 'true'


## Load Video Title dataset


#### Visualize details of dataset


In [7]:
## dataset_name="tonarie/Wayback-Data-Youtube-Homepage-Videos"

In [8]:

ds = load_dataset("csv", data_files="rlhf.csv", split="train")


In [9]:

ds


Dataset({
    features: ['title'],
    num_rows: 42899
})

In [10]:
ds[15:18]

{'title': ['SF International Film Festival Trailer',
  'Re-Enactment: Uncle Buck',
  'Evans Blue "Cold (But I\'m Still Here)" Music Video']}

In [11]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

In [12]:
def show_random_elements(dataset, num_examples=20):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    
    picks = []
    
    for _ in range( num_examples ):
        
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame( dataset[picks] )        ## indexing 10 picks 
    
    print(df)
    print(dataset.features.items())
    
    for column, typ in dataset.features.items():
        print(column)
        print(typ)
        print(ClassLabel)
        ## The isinstance() function returns True if the specified object 
        ## is of the specified type, otherwise False
        if isinstance(typ, ClassLabel):
            print("Hello")
            df[column] = df[column].transform(lambda i: typ.names[i])
            ## print(typ.names[i])
            
    display(HTML(df.to_html()))


In [13]:

show_random_elements(ds)


                                                title
0                 The Adventures of Batman and Rob...
1                                         TEXAS WINSS
2                              Angelina Jolie Cat Eye
3                                   motorcycle stunts
4     A Guy On Jets Side Lines Trips A Dolphin Player
5            UFC 117: Chael Sonnen Welcomes Advers...
6             SMACK/ URL Presents K-SHINE vs TAY ROCK
7            American Chopper Visits Jon & Kate Pl...
8                                             atchoum
9                           "The Soul of New Orleans"
10                          What Defines a Community?
11  Chris Colfer for The Trevor Project - It Gets ...
12  Eenie Meenie Bikini  (Eenie Meenie by Justin B...
13                       New GEICO Commercial - Piggy
14                                          Road Trip
15                           How much weight to lift?
16     Tea Party PSA (FreedomWorks) from D.C. Douglas
17          AT&T Don't Text 

,title
0,The Adventures of Batman and Rob...
1,TEXAS WINSS
2,Angelina Jolie Cat Eye
3,motorcycle stunts
4,A Guy On Jets Side Lines Trips A Dolphin Player
5,UFC 117: Chael Sonnen Welcomes Advers...
6,SMACK/ URL Presents K-SHINE vs TAY ROCK
7,American Chopper Visits Jon & Kate Pl...
8,atchoum
9,"""The Soul of New Orleans"""


In [14]:

tokenizer           = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token


In [15]:

def tokenize( sample ):
    sample["input_ids"] = tokenizer.encode( sample["title"]    )[: 20]
    sample["query"]     = tokenizer.decode( sample["input_ids"] )
    return sample


ds = ds.map(tokenize, batched=False)
ds.set_format(type="torch")
ds


Dataset({
    features: ['title', 'input_ids', 'query'],
    num_rows: 42899
})

In [16]:

ds[15:18]


{'title': ['SF International Film Festival Trailer',
  'Re-Enactment: Uncle Buck',
  'Evans Blue "Cold (But I\'m Still Here)" Music Video'],
 'input_ids': [tensor([20802,  4037, 13741, 11117, 36923]),
  tensor([ 3041,    12,  4834,   529,   434,    25, 23169, 13452]),
  tensor([15200,   504,  4518,   366, 34312,   357,  1537,   314,  1101,  7831,
           3423, 16725,  7849,  7623])],
 'query': ['SF International Film Festival Trailer',
  'Re-Enactment: Uncle Buck',
  'Evans Blue "Cold (But I\'m Still Here)" Music Video']}

In [17]:

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])
    


## Load GPT2

In [18]:

model     = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)

tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer.pad_token = tokenizer.eos_token


In [19]:

ppo_trainer = PPOTrainer(
                 config, 
                 model, 
                 ref_model, 
                 tokenizer, 
                 dataset=ds, 
                 data_collator=collator
)


## Load Reward Function

In [20]:

device = ppo_trainer.accelerator.device
device


device(type='cuda')

In [21]:

if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug

device

0

In [22]:

sentiment_pipe = pipeline("sentiment-analysis", model="christinacdl/XLM_RoBERTa-Clickbait-Detection-new", device=device)


In [23]:

text = "5 Things You Won't Believe"

sentiment_pipe(text, **sent_kwargs)


C:\Users\Jacob DeMuth\anaconda3\envs\py38_ITS530\lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'NOT', 'score': -5.232669353485107},
  {'label': 'CLICKBAIT', 'score': 5.39152193069458}]]

In [24]:

text = "A Dog Video"
sentiment_pipe(text, **sent_kwargs)


[[{'label': 'NOT', 'score': 1.463137149810791},
  {'label': 'CLICKBAIT', 'score': -1.969333291053772}]]

## Generation settings

In [25]:

gen_kwargs = {
         "min_length":   -1, 
         "top_k":       0.0, 
         "top_p":       1.0, 
         "do_sample":  True, 
         "pad_token_id": tokenizer.eos_token_id
}



## Optimize model

### Training loop

In [26]:
## more then 16 and it runs out of memory

output_min_length     = 4
output_max_length     = 8
output_length_sampler = LengthSampler(output_min_length, output_max_length)


In [27]:

generation_kwargs = {
    "min_length":     -1,
    "top_k":         0.0,
    "top_p":         1.0,
    "do_sample":    True,
    "pad_token_id": tokenizer.eos_token_id,
}


In [28]:
## ppo_trainer.config.steps = 100    ## 20,000
ppo_trainer.config.steps

20000

In [29]:

for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]
    
    print(query_tensors)
    print(len(query_tensors))
    if epoch == 1:
        break


1it [00:00,  3.50it/s]

[tensor([ 5308,   357, 16305,  1820,     8,   290, 32151, 12996, 13777,   366,
         2949,  3125,     1], device='cuda:0'), tensor([   1, 8164,   11, 6914,   11, 6914,   11, 6914,    1,  416, 2644],
       device='cuda:0'), tensor([   47,    83,   352,   366, 38667, 34198, 17011,   448, 11097,   220,
         3982,    72,   986], device='cuda:0'), tensor([45478,  6378,  1550, 10216,  1395,  5776], device='cuda:0'), tensor([   72, 23410], device='cuda:0'), tensor([   51,  1045, 33504,   717,  3427, 30424,  1067,   292,   986],
       device='cuda:0'), tensor([   42,  1404,    56, 19878, 18276, 33290,  5064, 30649,  3539,   402,
         4261,  6561,   357,  1845,  1118,     8,  2644], device='cuda:0'), tensor([   34,  1765,    84,  8211,   376,  1722, 15360], device='cuda:0'), tensor([   35,  2507,  3691,    13,  6183,   532,   383, 18692],
       device='cuda:0'), tensor([   34,  3861,    57,    56, 47200, 10705,     0,   357,  7556,  3768,
         6407,  8362,     8], device='cuda

In [30]:

for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]
    print(epoch)
    print(batch)
    print('*********************')
    print('*********************')
    print('*********************')
    print('*********************')
    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len                             = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response                            = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append( response.squeeze()[-gen_len:] )
    batch["response"] = [ tokenizer.decode(r.squeeze()) for r in response_tensors ]
    print(batch)
    if epoch == 1:
        break


0it [00:00, ?it/s]

0
{'input_ids': [tensor([31466,   298,  5438, 18448,    13], device='cuda:0'), tensor([37247,   868,   287,  9626,   370, 24929], device='cuda:0'), tensor([18332,   278,  2253,   350,  4090,   351,  4705, 33572],
       device='cuda:0'), tensor([40555,  2502,  5851, 12378,  1108], device='cuda:0'), tensor([   38,  4754,  5532,  2873,  1755,  9581,  1570,   422, 25762,  4391,
        24073], device='cuda:0'), tensor([ 2437,   284, 48887, 39801], device='cuda:0'), tensor([   42,  5910, 16754,   843, 13308,    64,  7459, 22244,  1114,   986],
       device='cuda:0'), tensor([42731, 11927,     0], device='cuda:0'), tensor([19555,  2257,  6833,  2310, 10891,    72,  5472], device='cuda:0'), tensor([15645,  4373,  2218,    13,  7039,  4908,   532, 11806,    64,  1629,
         2185], device='cuda:0'), tensor([   54,  7697, 11763,   317,  9897,    30], device='cuda:0'), tensor([24129,  2611, 32189,   513,    14,    18], device='cuda:0'), tensor([  464, 11165, 16089, 13388], device='cuda:0'), 

1it [00:37, 37.84s/it]

{'input_ids': [tensor([31466,   298,  5438, 18448,    13], device='cuda:0'), tensor([37247,   868,   287,  9626,   370, 24929], device='cuda:0'), tensor([18332,   278,  2253,   350,  4090,   351,  4705, 33572],
       device='cuda:0'), tensor([40555,  2502,  5851, 12378,  1108], device='cuda:0'), tensor([   38,  4754,  5532,  2873,  1755,  9581,  1570,   422, 25762,  4391,
        24073], device='cuda:0'), tensor([ 2437,   284, 48887, 39801], device='cuda:0'), tensor([   42,  5910, 16754,   843, 13308,    64,  7459, 22244,  1114,   986],
       device='cuda:0'), tensor([42731, 11927,     0], device='cuda:0'), tensor([19555,  2257,  6833,  2310, 10891,    72,  5472], device='cuda:0'), tensor([15645,  4373,  2218,    13,  7039,  4908,   532, 11806,    64,  1629,
         2185], device='cuda:0'), tensor([   54,  7697, 11763,   317,  9897,    30], device='cuda:0'), tensor([24129,  2611, 32189,   513,    14,    18], device='cuda:0'), tensor([  464, 11165, 16089, 13388], device='cuda:0'), te

1it [00:53, 53.90s/it]

{'input_ids': [tensor([2061,  284, 2822,  262, 1466,  287,  534, 1204,  532, 2644],
       device='cuda:0'), tensor([ 3041,    25,  2488, 47095,  1326,   499,  1647,   930,  5706, 43537],
       device='cuda:0'), tensor([43410, 16706,   406, 22436, 36557,  6416, 14213,   357, 13807,  1303,
           16,     8], device='cuda:0'), tensor([31443,    88, 12579], device='cuda:0'), tensor([45708,   805,   532,  9935,   338,  2892, 39795,  1475, 17040,    25,
          986], device='cuda:0'), tensor([2396,  314, 1138,  337, 9618,  329, 1103,   89,    0], device='cuda:0'), tensor([18858, 20492,  2199, 47714,   532,   366,   464, 37123,   437,   986],
       device='cuda:0'), tensor([43471,  2414,    25,   383, 27330,  4631, 10243], device='cuda:0'), tensor([ 9915, 26123,    25,  3232,   329,  1439,   352,  1222,   362],
       device='cuda:0'), tensor([   41, 29309, 19013,  3691,  4380,   319,   262,  8511],
       device='cuda:0'), tensor([46678, 43236,    42,   532, 31900,  7623,  3776, 440

In [31]:
batch.keys()

dict_keys(['input_ids', 'query', 'response'])

#### Compute Score

In [32]:

batch["query"]


['What to buy the women in your life -...',
 'Re: @knitmeapony | Old Spice',
 'Lindsay Lohan Needs Real Friends (Ep #1)',
 'Rainy Days',
 "Letterman - Dave's Monologue Excerpt:...",
 'So I met Miley for realz!',
 'Ann Marie Calhoun - "The Pretend...',
 'Mega64: The Beatles Rock Band',
 'Black Ops: Free for All 1 & 2',
 'Jumbo Jet vs People on the Beach',
 'STAR TREK - Angry Video Game Nerd',
 'Why do you watch the videos I ma...',
 'There Will Be Bud',
 'Actuación Insula Kampa, Pub Ágora 3.12.2010',
 'The Adventures of Batman and Rob...',
 'Float Like A Butterfly',
 "The Cacaman - Ain't Got Much of...",
 'Paladino threatens New York Post Editor',
 'kaylee',
 "ITN: Japan radiation 'causes no concern'",
 'The Hauntening',
 'Chicken!Chicken!',
 'PALE BLUE DOT',
 'Make a difference',
 'In the car with James May - Top Gear - BBC',
 'History Will Be Made - Radulov/Arnott',
 'Polish Police SCBA - Chopper',
 'IMAX in a basement',
 'Ned 1 Den 0',
 'BUTT MUSCLES & HEAVY BREATHING',
 'Giant House

In [33]:

batch["response"]


[' More\n\nBackground\n',
 ' Hothouse\n\nSubject:',
 ' Kristen Bell and her',
 '" was my favorite "Community"',
 ' Free View in iTunes\n',
 ' Lyrics by Hva Toll',
 '" January 8,',
 '). This Mesmorizer',
 ' Weapon DLC\n\nDecember 22',
 ' –\n\nPir',
 '\n\nYes, you read',
 '\n\nI found',
 's", and "Wra',
 '\n\nPico-Cola /',
 ' Free View in iTunes\n\n',
 ' 5.69 574 5',
 '\n\n30.46',
 "'s Journal\n\n",
 '.com\n\nWanna Know',
 ' says health ministry Read more',
 ' conjurers are a small',
 'Every time you make a "re',
 ' FOR ADULT M',
 ' with your Ether.\n\n',
 ' Radio 4 and BBC the Magazine',
 ' Would Be The Mess Hallers -',
 ' Rather at Risk - Cubism',
 ' office.\n\nHe projected the',
 ' Adam 4 Wisconsin Rlw 5',
 '\n\nThough we do not recommend',
 '粉SpiderWarrior',
 'this must be your spark',
 ' After Being Robbed, Buddy',
 ' Adult Ballshops\n\nW',
 ' Between now and November, more than',
 'igga"!Our exclusive',
 ' Freeling, Double Doctors, Transfer',
 ' Pokémon FireRed - Part 18 Pokémon',


In [34]:

texts = [ q + r for q, r in zip(batch["query"], batch["response"]) ]


In [35]:

texts


['What to buy the women in your life -... More\n\nBackground\n',
 'Re: @knitmeapony | Old Spice Hothouse\n\nSubject:',
 'Lindsay Lohan Needs Real Friends (Ep #1) Kristen Bell and her',
 'Rainy Days" was my favorite "Community"',
 "Letterman - Dave's Monologue Excerpt:... Free View in iTunes\n",
 'So I met Miley for realz! Lyrics by Hva Toll',
 'Ann Marie Calhoun - "The Pretend..." January 8,',
 'Mega64: The Beatles Rock Band). This Mesmorizer',
 'Black Ops: Free for All 1 & 2 Weapon DLC\n\nDecember 22',
 'Jumbo Jet vs People on the Beach –\n\nPir',
 'STAR TREK - Angry Video Game Nerd\n\nYes, you read',
 'Why do you watch the videos I ma...\n\nI found',
 'There Will Be Buds", and "Wra',
 'Actuación Insula Kampa, Pub Ágora 3.12.2010\n\nPico-Cola /',
 'The Adventures of Batman and Rob... Free View in iTunes\n\n',
 'Float Like A Butterfly 5.69 574 5',
 "The Cacaman - Ain't Got Much of...\n\n30.46",
 "Paladino threatens New York Post Editor's Journal\n\n",
 'kaylee.com\n\nWanna Know',
 "ITN

In [36]:

pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
pipe_outputs


[[{'label': 'NOT', 'score': 2.490553617477417},
  {'label': 'CLICKBAIT', 'score': -2.9494712352752686}],
 [{'label': 'NOT', 'score': 0.8059258460998535},
  {'label': 'CLICKBAIT', 'score': -1.302541971206665}],
 [{'label': 'NOT', 'score': -0.4256044924259186},
  {'label': 'CLICKBAIT', 'score': -0.1774040162563324}],
 [{'label': 'NOT', 'score': 1.0809476375579834},
  {'label': 'CLICKBAIT', 'score': -1.4847525358200073}],
 [{'label': 'NOT', 'score': 3.8315069675445557},
  {'label': 'CLICKBAIT', 'score': -4.299412727355957}],
 [{'label': 'NOT', 'score': 2.443516492843628},
  {'label': 'CLICKBAIT', 'score': -3.1101608276367188}],
 [{'label': 'NOT', 'score': 2.957875967025757},
  {'label': 'CLICKBAIT', 'score': -3.311002492904663}],
 [{'label': 'NOT', 'score': 3.8029301166534424},
  {'label': 'CLICKBAIT', 'score': -4.127135276794434}],
 [{'label': 'NOT', 'score': 2.576737880706787},
  {'label': 'CLICKBAIT', 'score': -3.088524580001831}],
 [{'label': 'NOT', 'score': 3.8692023754119873},
  {'l

In [37]:

rewards = [ torch.tensor(output[1]["score"]) for output in pipe_outputs]
rewards


[tensor(-2.9495),
 tensor(-1.3025),
 tensor(-0.1774),
 tensor(-1.4848),
 tensor(-4.2994),
 tensor(-3.1102),
 tensor(-3.3110),
 tensor(-4.1271),
 tensor(-3.0885),
 tensor(-4.2969),
 tensor(-1.0586),
 tensor(-3.7888),
 tensor(-3.8298),
 tensor(-4.2025),
 tensor(-3.9643),
 tensor(-1.1186),
 tensor(-3.8399),
 tensor(-4.5365),
 tensor(-3.2125),
 tensor(-4.4947),
 tensor(-3.8024),
 tensor(2.6146),
 tensor(-3.1890),
 tensor(-3.9331),
 tensor(-4.2898),
 tensor(-0.4648),
 tensor(-5.0327),
 tensor(-2.4057),
 tensor(-4.1132),
 tensor(4.0770),
 tensor(-3.2643),
 tensor(-2.7014),
 tensor(-4.4017),
 tensor(-3.9607),
 tensor(2.6887),
 tensor(2.3842),
 tensor(-4.0638),
 tensor(-2.1604),
 tensor(3.2921),
 tensor(-2.6237),
 tensor(-4.0694),
 tensor(-3.8018),
 tensor(-1.4327),
 tensor(-4.2847),
 tensor(2.5921),
 tensor(-3.6806),
 tensor(-4.0512),
 tensor(0.5040),
 tensor(-3.4761),
 tensor(-3.4257),
 tensor(-1.7957),
 tensor(-1.0368),
 tensor(-4.3142),
 tensor(-1.7293),
 tensor(-4.7236),
 tensor(-3.0547),

In [38]:

len(rewards)


128

In [39]:

for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]
    print(epoch)

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len                             = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response                            = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append( response.squeeze()[-gen_len:] )
    batch["response"] = [ tokenizer.decode(r.squeeze()) for r in response_tensors ]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [ torch.tensor(output[1]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(
                     query_tensors, 
                     response_tensors, 
                     rewards
    )
    ppo_trainer.log_stats(stats, batch, rewards)
    


0it [00:00, ?it/s]

0


1it [02:11, 131.94s/it]

1


2it [05:03, 155.05s/it]

...

333it [16:12:05, 178.47s/it]

333


334it [16:15:01, 177.67s/it]

334


335it [16:18:01, 175.17s/it]


In [40]:

torch.cuda.get_device_name(0)


'NVIDIA GeForce RTX 3080'

In [41]:

#### get a batch from the dataset
bs                 = 16
game_data          = dict()


In [42]:

game_data 


{}

In [43]:
dataset = ds

In [44]:

dataset.set_format("pandas")


In [45]:

df_batch           = dataset[:].sample(bs)
df_batch 


,title,input_ids,query
19960,SEXY DANCE TIME!,"[5188, 34278, 360, 19240, 20460, 0]",SEXY DANCE TIME!
4,SF International Film Festival Trailer,"[20802, 4037, 13741, 11117, 36923]",SF International Film Festival Trailer
38045,UNE GAFE DU GARDIEN DE TWENTE,"[41884, 402, 8579, 36, 35480, 402, 9795, 40, 1...",UNE GAFE DU GARDIEN DE TWENTE
39430,"GEICO's R. Lee Ermey, appearing on behalf of T...","[8264, 22707, 338, 371, 13, 5741, 5256, 1326, ...","GEICO's R. Lee Ermey, appearing on behalf of T..."
16686,"Michael Jackson's Neverland ""Ghost"" M...","[13256, 6612, 338, 7236, 1044, 366, 32001, 1, ...","Michael Jackson's Neverland ""Ghost"" M..."
18434,Anderson Cooper Destroys GOP Head Ove...,"[42991, 10382, 8145, 305, 893, 6796, 7123, 440...",Anderson Cooper Destroys GOP Head Ove...
27778,Annoying Orange: Back to the Fruiture,"[18858, 726, 278, 11942, 25, 5157, 284, 262, 2...",Annoying Orange: Back to the Fruiture
36617,Verloren - Luc Weegels Ft. Sandra Reemer,"[13414, 75, 29578, 532, 7598, 775, 1533, 1424,...",Verloren - Luc Weegels Ft. Sandra Reemer
27623,Lil Wayne - Light Up (Freestyle),"[43, 346, 13329, 532, 4401, 3205, 357, 20366, ...",Lil Wayne - Light Up (Freestyle)
5081,Front flips over 9 people Battle of the Eleme...,"[25886, 45971, 625, 860, 661, 220, 5838, 286, ...",Front flips over 9 people Battle of the Eleme...


In [46]:

game_data["query"] = df_batch["query"].tolist()
query_tensors      = df_batch["input_ids"].tolist()


In [47]:

response_tensors_ref, response_tensors = [], []


In [48]:

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    
    output  = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    
    
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)


In [49]:

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"]  = [tokenizer.decode(response_tensors[i]) for i in range(bs)]


In [50]:

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]


C:\Users\Jacob DeMuth\anaconda3\envs\py38_ITS530\lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
C:\Users\Jacob DeMuth\anaconda3\envs\py38_ITS530\lib\site-packages\transformers\pipelines\base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [51]:

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]


In [52]:

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results


,query,response (before),response (after),rewards (before),rewards (after)
0,SEXY DANCE TIME!,\n\nWarp,There's something that,-2.633893,4.688572
1,SF International Film Festival Trailer,| UNC Filmworks\n\n,\n\nHere's the trailer for,-4.350378,3.547578
2,UNE GAFE DU GARDIEN DE TWENTE,", ES BC.",This is what sort,-2.777253,3.222639
3,"GEICO's R. Lee Ermey, appearing on behalf of T...",Free View in iTunes\n\n,\n\nHere's the actual,-4.501187,4.406254
4,"Michael Jackson's Neverland ""Ghost"" M...","monument, Better",\n\nHere's,-1.522031,5.148306
5,Anderson Cooper Destroys GOP Head Ove...,Obama Is an Everyman,\n\nAnd here's,-2.775688,5.173042
6,Annoying Orange: Back to the Fruiture,ist\n\nThere is,", this is what they",-3.776184,4.450605
7,Verloren - Luc Weegels Ft. Sandra Reemer,- Il Divo 4 Remix,- here's the official cover,-3.925521,4.250594
8,Lil Wayne - Light Up (Freestyle),[VMAFT],. This song had been,-3.872023,2.891146
9,Front flips over 9 people Battle of the Eleme...,#2003Full video of Found,. Here's her response to,-4.306053,4.674588


In [53]:

print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())



mean:


rewards (before)   -3.028568
rewards (after)     3.801119
dtype: float64


median:


rewards (before)   -3.537325
rewards (after)     4.428430
dtype: float64

In [54]:

## model.save_pretrained(    "gpt2-imdb-pos-v2", push_to_hub=True)
## tokenizer.save_pretrained("gpt2-imdb-pos-v2", push_to_hub=True)

## model.save_pretrained(    "gpt2-imdb-pos-v2-jd", push_to_hub=False)
## tokenizer.save_pretrained("gpt2-imdb-pos-v2-jd", push_to_hub=False)



## Custom Querys

In [55]:
game_data["query"] = "Training a deep learning model with PyTorch for the Iris dataset (part 1)"

In [56]:
response_tensors_ref, response_tensors = [], []

In [57]:
#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    
    output  = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    
    
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"]  = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

C:\Users\Jacob DeMuth\anaconda3\envs\py38_ITS530\lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
C:\Users\Jacob DeMuth\anaconda3\envs\py38_ITS530\lib\site-packages\transformers\pipelines\base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,query,response (before),response (after),rewards (before),rewards (after)
0,Training a deep learning model with PyTorch fo...,\n\nOur hilarious,\n\nHere's,-2.797822,1.193287
1,Training a deep learning model with PyTorch fo...,":""Right now the world is at",. Here's the trailer:\n,2.515629,4.598798
2,Training a deep learning model with PyTorch fo...,ENTH PARTNUM,. This is actually,-3.473917,3.428020
3,Training a deep learning model with PyTorch fo...,18:02 < CodeShark,Here's what we happen to know,-4.319618,4.508544
4,Training a deep learning model with PyTorch fo...,\n\nPitch Perfect 2:,\n\nHere's What Happened,-3.657798,4.927411
5,Training a deep learning model with PyTorch fo...,MSNBC 4 hrs ago 0 149,There are one key reason why,-3.879203,4.196480
6,Training a deep learning model with PyTorch fo...,-Garden Contact This Page 13,. Here's the deal:\n,-4.060060,0.095456
7,Training a deep learning model with PyTorch fo...,- In All This Light (K,"!"" Here's the first twist.",-3.903644,4.397000
8,Training a deep learning model with PyTorch fo...,\n\nBillie,Here is his response,-1.879839,4.630973
9,Training a deep learning model with PyTorch fo...,. 5+ WAS,had this theory on,-3.194084,0.274144


In [58]:
game_data["query"] = "VPN Tun/Tap and sockets, routing, tunnels and TLS"

In [59]:
response_tensors_ref, response_tensors = [], []

In [60]:
#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    
    output  = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    
    
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"]  = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

C:\Users\Jacob DeMuth\anaconda3\envs\py38_ITS530\lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
C:\Users\Jacob DeMuth\anaconda3\envs\py38_ITS530\lib\site-packages\transformers\pipelines\base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,query,response (before),response (after),rewards (before),rewards (after)
0,"VPN Tun/Tap and sockets, routing, tunnels and TLS",Inspired by the Kyle and,These are actually how you should,-4.190799,4.860742
1,"VPN Tun/Tap and sockets, routing, tunnels and TLS",\n\nThe project opens,And Remember... Here's,-3.867090,4.867324
2,"VPN Tun/Tap and sockets, routing, tunnels and TLS",ISLES 249.,"-- ""JUST LOOK",-3.397221,-1.632957
3,"VPN Tun/Tap and sockets, routing, tunnels and TLS",‎ Appears in 20 books from,this is what it says…,-2.552716,4.267446
4,"VPN Tun/Tap and sockets, routing, tunnels and TLS",Free View in iTunes\n\n32,\nHere's one to take home,-4.506935,4.556669
5,"VPN Tun/Tap and sockets, routing, tunnels and TLS","\n\nCNN: ""#",\n\nHere's how,-2.520619,3.692731
6,"VPN Tun/Tap and sockets, routing, tunnels and TLS",t area in Pleasant,. This is just,-4.319515,0.147061
7,"VPN Tun/Tap and sockets, routing, tunnels and TLS",- Cobras And Frizz,\n\nHere's what he,-1.844394,4.618574
8,"VPN Tun/Tap and sockets, routing, tunnels and TLS",(Toure Remix,"""'It means he",-3.788618,1.738622
9,"VPN Tun/Tap and sockets, routing, tunnels and TLS",Pensacola 1928 Fighting,. This happened to a,-4.784885,4.367789


In [61]:
game_data["query"] = "Basic Ideas of Reinforcement Learning Through Human Feedbacks (RLHF)"

In [62]:
response_tensors_ref, response_tensors = [], []

In [63]:
#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    
    output  = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    
    
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"]  = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

C:\Users\Jacob DeMuth\anaconda3\envs\py38_ITS530\lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
C:\Users\Jacob DeMuth\anaconda3\envs\py38_ITS530\lib\site-packages\transformers\pipelines\base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,query,response (before),response (after),rewards (before),rewards (after)
0,Basic Ideas of Reinforcement Learning Through ...,This dance has very beautiful,""" We could be talking",0.224746,0.272968
1,Basic Ideas of Reinforcement Learning Through ...,Thumbs • TF,\n\nHere's your,-4.561722,-2.895056
2,Basic Ideas of Reinforcement Learning Through ...,EN KITCHEN Mich,". Come on, here",-2.794768,-1.882573
3,Basic Ideas of Reinforcement Learning Through ...,05:08\n\nJUSTICE,\n\nThis is the most obvious,-3.484382,4.189409
4,Basic Ideas of Reinforcement Learning Through ...,Free View in iTunes\n,\n\nHere's What,-4.295350,3.203177
5,Basic Ideas of Reinforcement Learning Through ...,John Dickerson and Maria,But he doesn't have,-4.254372,4.816490
6,Basic Ideas of Reinforcement Learning Through ...,ous Land Trapper |,", where you can find",-3.881571,-1.823045
7,Basic Ideas of Reinforcement Learning Through ...,- Do What U Want (,. This is the version of,-3.161539,-3.527610
8,Basic Ideas of Reinforcement Learning Through ...,[Drawcad,Here's what ended,-4.389244,4.763288
9,Basic Ideas of Reinforcement Learning Through ...,. The creator of the battle of,. Here's how it aired:,-3.230633,4.745613
